In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# ---------------------------
# 1. Extract Excel
# ---------------------------
file_path = r"C:\Users\higas\Downloads\generative-ai-data-architecture\Online_Retail.xlsx"
df = pd.read_excel(file_path)

# ---------------------------
# 2. Transform
# ---------------------------
df.drop_duplicates(inplace=True)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Country'] = df['Country'].str.upper()
df['Quantity'] = df['Quantity'].astype(int)
df['UnitPrice'] = df['UnitPrice'].astype(float)
df = df.dropna(subset=['CustomerID'])

# ---------------------------
# 3. Load to SQL (SQLite example)
# ---------------------------
engine = create_engine('sqlite:///online_retail.db', echo=False)

# Fact Table
df.to_sql('FactSales', con=engine, if_exists='replace', index=False)

# Dimension Tables
dim_customer = df[['CustomerID', 'Country']].drop_duplicates()
dim_customer.to_sql('DimCustomer', con=engine, if_exists='replace', index=False)

dim_product = df[['StockCode']].drop_duplicates()
dim_product['ProductDescription'] = None
dim_product.to_sql('DimProduct', con=engine, if_exists='replace', index=False)

dim_date = df[['InvoiceDate']].drop_duplicates()
dim_date['Day'] = dim_date['InvoiceDate'].dt.day
dim_date['Month'] = dim_date['InvoiceDate'].dt.month
dim_date['Year'] = dim_date['InvoiceDate'].dt.year
dim_date['Weekday'] = dim_date['InvoiceDate'].dt.day_name()
dim_date.to_sql('DimDate', con=engine, if_exists='replace', index=False)

# ---------------------------
# 4. Test SQL Query
# ---------------------------
query = """
SELECT f.InvoiceNo, f.StockCode, f.Quantity, f.UnitPrice, c.Country, d.Month
FROM FactSales f
JOIN DimCustomer c ON f.CustomerID = c.CustomerID
JOIN DimDate d ON f.InvoiceDate = d.InvoiceDate
LIMIT 10
"""
result = pd.read_sql(query, engine)
print(result)